In [ ]:
!pip install yfinance


In [23]:
import yfinance as yf

# Download VIX data
vix = yf.download('^VIX', start='2011-01-01', end='2024-01-01')
vix.to_csv('vix_2011_2023.csv')

# Download SPY data
spy = yf.download('SPY', start='2011-01-01', end='2024-01-01')
spy.to_csv('spy_2011_2023.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [24]:
!wget https://raw.githubusercontent.com/whit3rabbit/fear-greed-data/main/fear-greed-2011-2023.csv

--2024-09-22 14:59:47--  https://raw.githubusercontent.com/whit3rabbit/fear-greed-data/main/fear-greed-2011-2023.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42043 (41K) [text/plain]
Saving to: 'fear-greed-2011-2023.csv'

     0K .......... .......... .......... .......... .         100% 5.19M=0.008s

2024-09-22 14:59:47 (5.19 MB/s) - 'fear-greed-2011-2023.csv' saved [42043/42043]



In [25]:
import pandas as pd

# Load the datasets
vix_data = pd.read_csv('vix_2011_2023.csv')
spy_data = pd.read_csv('spy_2011_2023.csv')
fear_greed_data = pd.read_csv('fear-greed-2011-2023.csv')

# Standardizing the date format in the Fear & Greed dataset
fear_greed_data['Date'] = pd.to_datetime(fear_greed_data['Date'])
vix_data['Date'] = pd.to_datetime(vix_data['Date'])
spy_data['Date'] = pd.to_datetime(spy_data['Date'])

# Merging the datasets on the 'Date' column
combined_data = pd.merge(pd.merge(vix_data, spy_data, on='Date', how='left', suffixes=('_VIX', '_SPY')), fear_greed_data, on='Date', how='left')

# Removing the dates with missing VIX and SPY data
cleaned_combined_data = combined_data.dropna(subset=['Open_VIX', 'Open_SPY'])

# Removing the 'Volume_VIX' column from the cleaned dataset since its empty
cleaned_combined_data = cleaned_combined_data.drop(columns=['Volume_VIX'])


In [19]:
fear_greed_data.isna().sum()

Date          0
Fear Greed    0
dtype: int64

In [20]:
cleaned_combined_data.isna().sum()

Date              0
Open_VIX          0
High_VIX          0
Low_VIX           0
Close_VIX         0
Adj Close_VIX     0
Open_SPY          0
High_SPY          0
Low_SPY           0
Close_SPY         0
Adj Close_SPY     0
Volume_SPY        0
Fear Greed       22
dtype: int64

In [28]:
#Fill Na with the average of the FG Index day before and day after the missing period

cleaned_combined_data.loc[cleaned_combined_data['Fear Greed'].isna(), 'Fear Greed'] = 60

In [29]:
# Saving the cleaned data to a CSV file
cleaned_combined_data.to_csv('spy_vix_fear_greed_2011_2023.csv', index=False)